In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
from math import floor

In [4]:
def generate_random_matrix(size):
    return np.random.uniform(1e-8, 1.0, (size, size))

## Rekurencyjna eliminacja Gaussa

In [156]:
def recursive_gaussian_elimination(A, b):

    n = A.shape[0]

    if n == 1:
        return np.array([b[0] / A[0, 0]])
    
    mid = n // 2
    A11 = A[:mid, :mid]
    A12 = A[:mid, mid:]
    A21 = A[mid:, :mid]
    A22 = A[mid:, mid:]

    b1 = b[:mid]
    b2 = b[mid:]

    b_prime = b2 - A21 @ np.linalg.solve(A11, b1)

    S = A22 - A21 @ np.linalg.solve(A11, A12)

    x2 = recursive_gaussian_elimination(S, b_prime)

    x1 = np.linalg.solve(A11, b1 - A12 @ x2)
    
    x = np.concatenate((x1, x2))

    return x

In [161]:
start = 1
stop = 100

for test_num in range(start, stop+1):
    A = generate_random_matrix(test_num)
    b = np.random.rand(test_num)

    while np.linalg.det(A) == 0:
        A = generate_random_matrix(test_num)

    x_recursive = recursive_gaussian_elimination(A, b)
    x_numpy = np.linalg.solve(A, b)
    
    if np.allclose(x_recursive, x_numpy):
        # print(f"Test passed for random A of size {test_num}x{test_num}")
        continue
    else:
        print(f"Test failed for random square matrices of size {test_num}x{test_num}")
        print("Solution using recursive inverse:")
        print(x_recursive)

        print("Expected solution:")
        print(x_numpy)
        break
else:
    print(f"Test passed for random square matrices of sizes {start}x{start} to {stop}x{stop}")

Test passed for random square matrices of sizes 1x1 to 100x100


## Rekurencyjne odwracanie macierzy

In [55]:
def recursive_matrix_inverse(A):
    n = A.shape[0]

    if n == 1:
        return np.array([[1 / A[0, 0]]])
    
    mid = n // 2
    A11 = A[:mid, :mid]
    A12 = A[:mid, mid:]
    A21 = A[mid:, :mid]
    A22 = A[mid:, mid:]

    A11_inv = recursive_matrix_inverse(A11)

    S = A22 - A21 @ A11_inv @ A12

    S_inv = recursive_matrix_inverse(S)
    B11 = A11_inv + A11_inv @ A12 @ S_inv @ A21 @ A11_inv
    B12 = -A11_inv @ A12 @ S_inv
    B21 = -S_inv @ A21 @ A11_inv
    B22 = S_inv

    A_inverse = np.vstack((
        np.hstack((B11, B12)),
        np.hstack((B21, B22))
        ))

    return A_inverse

In [140]:
start = 1
stop = 50

for test_num in range(start, stop+1):
    matrix = generate_random_matrix(test_num)

    while np.linalg.det(matrix) == 0:
        matrix = generate_random_matrix(test_num)
    
    matrix_inv_recursive = recursive_matrix_inverse(matrix)
    matrix_inv_numpy = np.linalg.inv(matrix)

    if np.allclose(matrix_inv_recursive, matrix_inv_numpy):
        # print(f"Test passed for random matrix of size {test_num}x{test_num}")
        continue
    else:
        print(f"Test failed for random matrix of size {test_num}x{test_num}")
        print("Solution using recursive inverse:")
        print(matrix_inv_recursive)

        print("Expected solution:")
        print(matrix_inv_numpy)
        break
else:
    print(f"Test passed for random square matrices of sizes {start}x{start} to {stop}x{stop}")

Test passed for random matrices of sizes 1x1 to 50x50 


## Rekurencyjna LU faktoryzacja 

In [141]:
def lu_recursive(A):
    n = A.shape[0]

    if n == 1:
        return np.array([[1]]), np.array([[A[0,0]]])
    
    mid = (n + 1) // 2
    A11 = A[:mid, :mid]
    A12 = A[:mid, mid:]
    A21 = A[mid:, :mid]
    A22 = A[mid:, mid:]

    L11, U11 = lu_recursive(A11)
    U11_inv = np.linalg.inv(U11)
    L11_inv = np.linalg.inv(L11)

    L21 = A21 @ U11_inv
    U12 = L11_inv @ A12 

    if A22.size > 0:
        S = A22 - L21 @ U12
        Ls, Us = lu_recursive(S)
    else:
        Ls, Us = np.empty_like(L21), np.empty_like(U12)

    L = np.block([
        [L11, np.zeros((mid, A.shape[1] - mid))],
        [L21, Ls]
    ])

    U = np.block([
        [U11, U12],
        [np.zeros((A.shape[0] - mid, mid)), Us]
    ])

    return L, U

In [142]:
start = 1
stop = 50

for test_num in range(start, stop+1):
    matrix = generate_random_matrix(test_num)
    
    L, U = lu_recursive(matrix)
    matrix_reconstructed = L @ U

    if np.allclose(matrix, matrix_reconstructed):
        # print(f"Test passed for random matrix of size {test_num}x{test_num}")
        continue

    else:
        print(f"Test failed for random matrix of size {test_num}x{test_num}")
        print("Solution using lu recursive:")
        print(matrix_inv_recursive)

        print("Expected solution:")
        print(matrix_inv_numpy)
        break
else:
    print(f"Test passed for random square matrices of sizes {start}x{start} to {stop}x{stop}")

Test passed for random square matrices of sizes 1x1 to 50x50


## Rekurencyjne obliczanie wyznacznika macierzy

In [110]:
def recursive_determinant_lu(A):
    L, U = lu_recursive(A)
    return np.prod(np.diag(L)) * np.prod(np.diag(U))

In [152]:
start = 1
stop = 50

for test_num in range(start, stop+1):
    A = generate_random_matrix(test_num)

    A_det_recursive = recursive_determinant_lu(A)
    A_det_numpy = np.linalg.det(A)    
    
    if np.allclose(A_det_recursive, A_det_numpy):
        # print(f"Test passed for random A of size {test_num}x{test_num}")
        continue
    else:
        print(f"Test failed for random square matrices of size {test_num}x{test_num}")
        print("Solution using recursive inverse:")
        print(A_det_recursive)

        print("Expected solution:")
        print(A_det_numpy)
        break
else:
    print(f"Test passed for random square matrices of sizes {start}x{start} to {stop}x{stop}")

Test passed for random square matrices of sizes 1x1 to 50x50
